Training a Conv LSTM based Model for Image Segmentation

In [1]:
# !conda env create -f environment.yaml

In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms
from tqdm import tqdm

In [3]:
# !gdown https://drive.google.com/uc?id=1I64DYabWlUU4D4ktAS8IMSrQxlaGJIKi

In [4]:
# Following takes a really long time.
# !unzip Dataset_Student.zip &> /dev/null

In [5]:
# Load image dataset
train_dataset = datasets.ImageFolder(root='Dataset_Student/train', 
                                     transform=transforms.ToTensor())
val_dataset = datasets.ImageFolder(root='Dataset_Student/val',
                                   transform=transforms.ToTensor())

In [6]:
# np.unique([entry[0].shape for entry in train_dataset], return_counts=True)
# all the images have the same size

In [7]:
mask = np.load('Dataset_Student/train/video_0/mask.npy')
print(mask.shape)
# mask also has the same shape as the images

(22, 160, 240)


In [11]:
# Creating a custom dataset for frames
class FrameDataset(Dataset):
    def __init__(self, root_folder, transforms=None):
        self.root_folder = root_folder
        self.transforms = transforms
        
        # Get all the folders in the root folder
        self.video_folders = os.listdir(root_folder)
        self.video_folders.sort()
        self.video_folders = [os.path.join(root_folder, folder) for folder in self.video_folders]
        self.video_folders = [folder for folder in self.video_folders if os.path.isdir(folder)]
        
        # Get all the mask files
        self.video_masks = [os.path.join(folder, 'mask.npy') for folder in self.video_folders]
        
    def __len__(self):
        return len(self.video_folders)
    
    def __getitem__(self, idx):
        # Get the folder
        folder = self.video_folders[idx]
        
        # Get all the images in the folder
        images = os.listdir(folder)
        images.sort()
        images = [os.path.join(folder, image) for image in images if image.endswith('.png')]
        images = [image for image in images if os.path.isfile(image)]
        
        # Load the images
        images = [plt.imread(image) for image in images]
        images = np.array(images)
        images = np.moveaxis(images, -1, 1)
        
        # Load the mask
        mask = np.load(self.video_masks[idx])
        
        # Apply the transformation
        if self.transforms:
            images = self.transforms(images)
            mask = self.transforms(mask)
        
        return {"images": images, "mask": mask}

transformations = transforms.Compose([
    torch.from_numpy,
    transforms.Resize(3),
    ])

train_dataset = FrameDataset(root_folder='Dataset_Student/train', transforms=transformations)
val_dataset = FrameDataset(root_folder='Dataset_Student/val', transforms=transformations)

In [12]:
train_dataset[0]['images'].shape, train_dataset[0]['mask'].shape

/home/ss14412/.conda/envs/pred/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


(torch.Size([22, 3, 3, 4]), torch.Size([22, 3, 4]))

In [13]:
len(train_dataset), len(val_dataset)

(1000, 1000)

In [14]:
# creating the dataloaders
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=4, shuffle=True)

In [15]:
for batch in train_loader:
    print(batch['images'].shape)
    print(batch['mask'].shape)
    break

torch.Size([4, 22, 3, 3, 4])
torch.Size([4, 22, 3, 4])


## Convolutional LSTM implementation

Code from: https://github.com/ndrplz/ConvLSTM_pytorch

In [16]:
class ConvLSTMCell(nn.Module):

    def __init__(self, input_dim, hidden_dim, kernel_size, bias):
        """
        Initialize ConvLSTM cell.

        Parameters
        ----------
        input_dim: int
            Number of channels of input tensor.
        hidden_dim: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        bias: bool
            Whether or not to add the bias.
        """

        super(ConvLSTMCell, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        self.kernel_size = kernel_size
        self.padding = kernel_size[0] // 2, kernel_size[1] // 2
        self.bias = bias

        self.conv = nn.Conv2d(in_channels=self.input_dim + self.hidden_dim,
                              out_channels=4 * self.hidden_dim,
                              kernel_size=self.kernel_size,
                              padding=self.padding,
                              bias=self.bias)

    def forward(self, input_tensor, cur_state):
        h_cur, c_cur = cur_state

        combined = torch.cat([input_tensor, h_cur], dim=1)  # concatenate along channel axis

        combined_conv = self.conv(combined)
        cc_i, cc_f, cc_o, cc_g = torch.split(combined_conv, self.hidden_dim, dim=1)
        i = torch.sigmoid(cc_i)
        f = torch.sigmoid(cc_f)
        o = torch.sigmoid(cc_o)
        g = torch.tanh(cc_g)

        c_next = f * c_cur + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next

    def init_hidden(self, batch_size, image_size):
        height, width = image_size
        return (torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device),
                torch.zeros(batch_size, self.hidden_dim, height, width, device=self.conv.weight.device))

In [17]:
class ConvLSTM(nn.Module):

    """

    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.

    Input:
        A tensor of size B, T, C, H, W or T, B, C, H, W
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers,
                 batch_first=False, bias=True, return_all_layers=False):
        super(ConvLSTM, self).__init__()

        self._check_kernel_size_consistency(kernel_size)

        # Make sure that both `kernel_size` and `hidden_dim` are lists having len == num_layers
        kernel_size = self._extend_for_multilayer(kernel_size, num_layers)
        hidden_dim = self._extend_for_multilayer(hidden_dim, num_layers)
        if not len(kernel_size) == len(hidden_dim) == num_layers:
            raise ValueError('Inconsistent list length.')

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.kernel_size = kernel_size
        self.num_layers = num_layers
        self.batch_first = batch_first
        self.bias = bias
        self.return_all_layers = return_all_layers

        cell_list = []
        for i in range(0, self.num_layers):
            cur_input_dim = self.input_dim if i == 0 else self.hidden_dim[i - 1]

            cell_list.append(ConvLSTMCell(input_dim=cur_input_dim,
                                          hidden_dim=self.hidden_dim[i],
                                          kernel_size=self.kernel_size[i],
                                          bias=self.bias))

        self.cell_list = nn.ModuleList(cell_list)

    def forward(self, input_tensor, hidden_state=None):
        """

        Parameters
        ----------
        input_tensor: todo
            5-D Tensor either of shape (t, b, c, h, w) or (b, t, c, h, w)
        hidden_state: todo
            None. todo implement stateful

        Returns
        -------
        last_state_list, layer_output
        """
        if not self.batch_first:
            # (t, b, c, h, w) -> (b, t, c, h, w)
            input_tensor = input_tensor.permute(1, 0, 2, 3, 4)

        b, _, _, h, w = input_tensor.size()

        # Implement stateful ConvLSTM
        if hidden_state is not None:
            raise NotImplementedError()
        else:
            # Since the init is done in forward. Can send image size here
            hidden_state = self._init_hidden(batch_size=b,
                                             image_size=(h, w))

        layer_output_list = []
        last_state_list = []

        seq_len = input_tensor.size(1)
        cur_layer_input = input_tensor

        for layer_idx in range(self.num_layers):

            h, c = hidden_state[layer_idx]
            output_inner = []
            for t in range(seq_len):
                h, c = self.cell_list[layer_idx](input_tensor=cur_layer_input[:, t, :, :, :],
                                                 cur_state=[h, c])
                output_inner.append(h)

            layer_output = torch.stack(output_inner, dim=1)
            cur_layer_input = layer_output

            layer_output_list.append(layer_output)
            last_state_list.append([h, c])

        if not self.return_all_layers:
            layer_output_list = layer_output_list[-1:]
            last_state_list = last_state_list[-1:]

        return layer_output_list, last_state_list

    def _init_hidden(self, batch_size, image_size):
        init_states = []
        for i in range(self.num_layers):
            init_states.append(self.cell_list[i].init_hidden(batch_size, image_size))
        return init_states

    @staticmethod
    def _check_kernel_size_consistency(kernel_size):
        if not (isinstance(kernel_size, tuple) or
                (isinstance(kernel_size, list) and all([isinstance(elem, tuple) for elem in kernel_size]))):
            raise ValueError('`kernel_size` must be tuple or list of tuples')

    @staticmethod
    def _extend_for_multilayer(param, num_layers):
        if not isinstance(param, list):
            param = [param] * num_layers
        return param

In [18]:
class seq2seq(nn.Module):
    def __init__(self, input_dim, hidden_dim, kernel_size, num_layers, batch_first, bias, return_all_layers):
        super(seq2seq, self).__init__()
        self.conv_lstm = ConvLSTM(input_dim, hidden_dim, kernel_size, num_layers, batch_first, bias, return_all_layers)
        self.fc = nn.Linear(hidden_dim[-1], 49)
    
    def forward(self, x):
        x, _ = self.conv_lstm(x)
        x = x[0].permute(0, 1, 3, 4, 2)
        x = self.fc(x)
        return x

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [21]:
class EarlyStopper:
    def __init__(self, patience=1, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.min_validation_loss = np.inf

    def early_stop(self, validation_loss):
        if validation_loss < self.min_validation_loss:
            self.min_validation_loss = validation_loss
            self.counter = 0
        elif validation_loss > (self.min_validation_loss + self.min_delta):
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

In [22]:
from torch.optim import lr_scheduler

model = seq2seq(input_dim=3,
             hidden_dim=[8, 8, 16],
             kernel_size=(3, 3),
             num_layers=3,
             batch_first=True,
             bias=True,
             return_all_layers=False).to(device)

# hyperparameters
criteria = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)
early_stopper = EarlyStopper(patience=10, min_delta=0.001)
epochs = 10

In [24]:
# Load checkpoint
checkpoint = torch.load('best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

In [38]:
def train(model, train_loader, val_loader, optimizer, criteria, epochs, device, early_stopper):
    model.train()
    for epoch in range(epochs):
        losses = []
        for batch_idx, data in enumerate(tqdm(train_loader)):
            images, masks = data['images'].to(device), data['mask'].to(device)
            optimizer.zero_grad()
            output = model(images).squeeze(-1)
            loss = criteria(output.argmax(dim=-1).float(), masks.float())
            loss.requires_grad = True
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        val_loss, mean_acc = validate(model, val_oader, criteria)
        print(f"Epoch: {epoch+1}/{epochs}, Loss: {np.mean(losses)}, Val Loss: {val_loss}, Mean Acc: {mean_acc}")
        
        
        
        if early_stopper.early_stop(val_loss):
            print("Early stopping")
            break
            

In [204]:
# train(model, train_loader, optimizer, criteria, epochs)

100%|██████████| 250/250 [06:40<00:00,  1.60s/it]

Epoch: 1/1, Loss: 1187.166111328125


In [33]:
def validate(model, val_loader, criteria):
    model.eval()
    losses, accuracies = [], []
    with torch.no_grad():
        for batch_idx, data in enumerate(tqdm(val_loader)):
            images, masks = data['images'].to(device), data['mask'].to(device)
            output = model(images).squeeze(-1)
            loss = criteria(output.argmax(dim=-1).float(), masks.float())
            prediction = (output.argmax(dim=-1) == masks).sum().item()
            accuracies.append(prediction/masks.numel())
            losses.append(loss.item())
    return np.mean(losses), np.mean(accuracies)

In [34]:
val_loss, mean_acc = validate(model, val_dataloader, criteria)

  0%|          | 0/250 [00:00<?, ?it/s]/home/ss14412/.conda/envs/pred/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
100%|██████████| 250/250 [00:58<00:00,  4.29it/s]

Validation Loss: 1186.4833354492187, Mean Accuracy: 0.0012727272727272724


In [37]:
print(f"Validation Loss: {val_loss:.2f}, Mean Accuracy: {mean_acc:.4f}")

Validation Loss: 1186.48, Mean Accuracy: 0.0013


In [205]:
torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': 1187.166111328125,
            }, 'best_model.pt')

# load model from here --> 
# https://pytorch.org/tutorials/recipes/recipes/saving_and_loading_a_general_checkpoint.html